# Ejercicio final de la semana 1

Para demostrar que estás familiarizado con la API de OpenAI y también con Ollama, crea una herramienta que responda a una pregunta técnica
y la explique. ¡Esta es una herramienta que podrás usar durante el curso!

In [8]:
# imports
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from dotenv import load_dotenv
import os

# constantes

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
openai = OpenAI()

# Configura el entorno
# Cargamos las variables del fichero .env
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

system_prompt="Eres un experto programador en Python 3.12 y vas a responderme todas las dudas que tenga con respcecto \
a fragmentos de código que te de, así como explicarme el código en cuestión. \
tambien me explicaras terminos que no entienda de la programación"

def obtain_user_prompt():
    user_question = [] 
    line = input("Introduce un prompt que termine con una línea vacía: ")
    while line:
        user_question.append(line)
        line = input() 

    user_question = "\n".join(user_question)
    return user_question


# Haz que gpt-4o-mini responda, con streaming
def stream_response():
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": obtain_user_prompt()}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)


In [9]:
stream_response()

Introduce un prompt que termine con una línea vacía:  Explica qué hace este código y por qué:
 yield from {book.get("author") for book in books if book.get("author")}
 


Este fragmento de código utiliza **`yield from`** en combinación con una **comprensión de conjunto** (set comprehension) para generar una secuencia de valores. Vamos a desglosar el código por partes:

1. **Comprensión de conjunto**: 
   python
   {book.get("author") for book in books if book.get("author")}
   
   - Aquí, se está creando un conjunto que incluye los autores de los libros almacenados en la variable `books`.
   - `books` se espera que sea una lista (o cualquier iterable) de diccionarios, donde cada diccionario representa un libro y tiene, al menos, la clave `"author"`.
   - `book.get("author")` intenta obtener el valor asociado con la clave `"author"` de cada diccionario (libro). El método `get()` devuelve `None` si la clave no está presente.
   - La expresión `if book.get("author")` se usa como un filtro para incluir solo aquellos libros que tienen un autor definido. Por lo tanto, solo se incluirán los valores de autor que no sean `None` o que no estén vacíos.

2. **`yield from`**:
   python
   yield from { ... }
   
   - La palabra clave `yield` se utiliza dentro de una **función generadora** para devolver un valor y pausar la ejecución de la función, permitiendo reanudarla más tarde. La palabra clave `from` se utiliza aquí para delegar en otro generador iterable.
   - `yield from` permite que la función generadora produzca todos los valores de un iterable (en este caso, el conjunto de autores) uno por uno.
   - En este contexto específico, `yield from` toma cada autor del conjunto creado y lo devuelve como resultado de la función generadora que contiene este código.

### Resumen:
Este código está diseñado para devolver, uno a uno, todos los autores de los libros en la lista `books`, pero solo aquellos que tienen un autor especificado. Utiliza un conjunto (`{ ... }`) para asegurarse de que los autores devueltos sean únicos (sin duplicados).

Si en el caso de que el conjunto de libros `books` contiene libros con diferentes autores, este fragmento de código te permitirá iterar sobre la lista de autores únicos de manera eficiente y limpia.

In [ ]:
# Aquí está la pregunta; escribe sobre esto para preguntar algo nuevo

question = """
Explica qué hace este código y por qué:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [11]:
# imports
from IPython.display import Markdown, display, update_display
import requests

# constantes

MODEL_LLAMA = 'llama3.2'
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}


def obtain_user_prompt():
    user_question = [] 
    line = input("Introduce un prompt que termine con una línea vacía: ")
    while line:
        user_question.append(line)
        line = input() 

    user_question = "\n".join(user_question)
    return user_question


# Haz que Llama 3.2 responda
def llama_tutor():

    messages=[{"role": "user", "content": obtain_user_prompt()}]
    payload = {
        "model": MODEL_LLAMA,
        "messages": messages,
        "stream": False
    }
    
    response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
    print(response.json()['message']['content'])

In [12]:
llama_tutor()

Introduce un prompt que termine con una línea vacía:  Explica qué hace este código y por qué:
 yield from {book.get("author") for book in books if book.get("author")}
 


**Análisis del Código**

El código proporcionado utiliza una técnica común en Python llamada "generador" (en inglés, "generator"). Un generador es un objeto que permite iterar sobre una secuencia de valores sin tener que almacenarlos todos en memoria a la vez.

```python
yield from {book.get("author") for book in books if book.get("author")}
```

**Cómo funciona el código**

Aquí hay una explicación detallada de cómo funciona:

1. **Definición del generador**: `yield from` es un operador que permite pasar el resultado de una secuencia (en este caso, un dict en forma de conjunto) a otro.
2. **Generación del conjunto**: El conjunto `{book.get("author") for book in books if book.get("author")}` se genera utilizando la sintaxis de listas generativas (`[]` y `for`). En esta secuencia:
 * `for book in books`: Itera sobre cada libro en la lista `books`.
 * `if book.get("author")`: Filtera los libros que tengan una clave `"author"` existente.
 * `book.get("author")`: Extrae el valor asociado a